<a href="https://colab.research.google.com/github/revathithollikonda/gitdemo/blob/main/Sequence_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
  with open('/content/INFY20002008.csv', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


In [ ]:
x = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 100)           1316600   
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 13166)             1988066   
                                                                 
Total params: 3455266 (13.18 MB)
Trainable params: 3455266 (13.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=10, verbose=1)

Epoch 1/10
1904/1904 [==============================] - 213s 111ms/step - loss: 5.6796 - accuracy: 0.1857
Epoch 2/10
1904/1904 [==============================] - 208s 109ms/step - loss: 4.9836 - accuracy: 0.2067
Epoch 3/10
1904/1904 [==============================] - 210s 110ms/step - loss: 4.7432 - accuracy: 0.2139
Epoch 4/10
1904/1904 [==============================] - 209s 110ms/step - loss: 4.4696 - accuracy: 0.2273
Epoch 5/10
1904/1904 [==============================] - 209s 110ms/step - loss: 4.2074 - accuracy: 0.2446
Epoch 6/10
1904/1904 [==============================] - 207s 109ms/step - loss: 3.9215 - accuracy: 0.2693
Epoch 7/10
1904/1904 [==============================] - 199s 104ms/step - loss: 3.6272 - accuracy: 0.2968
Epoch 8/10
1904/1904 [==============================] - 197s 103ms/step - loss: 3.3281 - accuracy: 0.3340
Epoch 9/10
1904/1904 [==============================] - 203s 106ms/step - loss: 3.0388 - accuracy: 0.3819
Epoch 10/10
1904/1904 [=======================

In [ ]:
seed_text = "hello there!"
next_words = 40

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)